<a href="https://www.kaggle.com/code/mdansarullahtohor/text-preprocessing-for-nlp?scriptVersionId=257944297" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


Text preprocessing step

1.lowercasing/uppercasing all data

2.Remove html tags if html tags are present in data ( using python regular expression--> regex)

3.Remove urls 

4.Remove punctuation

5.chat word treatment(lmao,asap, etc)

6.Spelling Correction

7.Removing Stop words.a, the, of, are,my etc ( dont need to remove stop words when we perform parts of speech tagging )

using nltk we can remove stop words.(from nltk.corpus import stopwords)

8.Remove Emojis or converts emojis into its corresponding meaning.

9.Tokenization( sentense level tokenization and word level tokenization)
Tokenization is used to convert sentense as number like finding the number of unique words in a sentense.. example:
i am a student---> where number of unique word is 4..

10.Stemming(Using NLTK)

11.Lemmatization( kind of stemming but better bacause it search is it belong to  the language. in lemmatization root word is calles Lemma.) 
using wordnetlemmatizer we can perform lemmatization

In [2]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


1.Our task is to make lowercase of all reviews column description


In [4]:
df['review']=df['review'].str.lower()

In [5]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


2. Remove Html tags 

In [6]:
import re
def remove_html_tags(text):
    pattern=re.compile('<.*?>')
    return pattern.sub(r'',text)
    

In [7]:
text="<p><b>Hello</b> <i>World</i></p>"

In [8]:
remove_html_tags(text)

'Hello World'

now use this above function to remove html tags from our datt frame (df)

In [9]:
df['review']=df['review'].apply(remove_html_tags)

In [10]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


3.Remove urls from text


In [11]:
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [12]:
text= 'check my note book https://www.kaggle.com/code/mdansarullahtohor/text-preprocessing/edit '

In [13]:
remove_url(text)

'check my note book  '

In [14]:
df['review']=df['review'].apply(remove_url)

4.Remove punctuation


In [15]:
import string,time

In [16]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
exclude='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
def remove_punctuation(text):
    for punc in exclude:
        
       text=text.replace(punc,'')
    return text


In [19]:
text1='String. with. punctuation?'

In [20]:
start=time.time()
print(remove_punctuation(text1))
time1=time.time()-start
print (time1*50000)

String with punctuation
10.883808135986328


this above process of punctuation remove takes lot time for big dataset. instead of doing this we can use another function 

In [21]:
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [22]:
start=time.time()
print(remove_punctuation(text1))
time2=time.time()-start


String with punctuation


In [23]:
time1/time2

1.547457627118644

it is faster 1.744 times faster than previous

In [24]:
df['review']=df['review'].apply(remove_punc)

In [25]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,i am a catholic taught in parochial elementary...,negative
49998,im going to have to disagree with the previous...,negative


5. remove chat word like lmao,asap,dm,lol etc

In [26]:
chat_dict = {
    "afk": "away from keyboard",
    "afaik": "as far as i know",
    "asap": "as soon as possible",
    "atm": "at the moment",
    "bff": "best friends forever",
    "brb": "be right back",
    "bruh": "bro / brother",
    "btw": "by the way",
    "cya": "see you",
    "dm": "direct message",
    "fomo": "fear of missing out",
    "ftw": "for the win",
    "fyi": "for your information",
    "gg": "good game",
    "gl": "good luck",
    "gtg": "got to go",
    "hbu": "how about you",
    "idc": "i don’t care",
    "idk": "i don’t know",
    "ily": "i love you",
    "imho": "in my humble opinion",
    "imo": "in my opinion",
    "ikr": "i know right",
    "j/k": "just kidding",
    "jk": "just kidding",
    "lmao": "laughing my ass off",
    "lol": "laugh out loud",
    "np": "no problem",
    "nvm": "never mind",
    "omg": "oh my god",
    "omgosh": "oh my gosh",
    "rofl": "rolling on the floor laughing",
    "rn": "right now",
    "smh": "shaking my head",
    "tbh": "to be honest",
    "tmi": "too much information",
    "ttyl": "talk to you later",
    "uwu": "cute face",
    "wyd": "what are you doing",
    "wtf": "what the f***",
    "yw": "you’re welcome",
    "zzz": "sleeping / bored",
    "asl": "age, sex, location",
    "bc": "because",
    "bcoz": "because",
    "bf": "boyfriend",
    "gf": "girlfriend",
    "cba": "can’t be arsed",
    "cu": "see you",
    "cuz": "because",
    "dc": "disconnect / direct current",
    "dw": "don’t worry",
    "ftl": "for the loss",
    "gr8": "great",
    "gtfo": "get the f*** out",
    "hf": "have fun",
    "irl": "in real life",
    "lmk": "let me know",
    "ly": "love you",
    "mb": "my bad",
    "nsfw": "not safe for work",
    "omw": "on my way",
    "plz": "please",
    "ppl": "people",
    "sq": "status quo",
    "sry": "sorry",
    "tgif": "thank god it’s friday",
    "wth": "what the hell",
    "xoxo": "hugs and kisses",
    "yolo": "you only live once",
    "n/a": "not applicable",
    "obv": "obviously",
    "qq": "crying / whining",
    "wbu": "what about you",
    "ttfn": "ta ta for now",
    "bbl": "be back later",
    "cwtch": "hug / cuddle",
    "fml": "f*** my life",
    "hmu": "hit me up",
    "irl": "in real life",
    "jk": "just kidding",
    "lmk": "let me know",
    "ns": "nice shot",
    "otp": "one true pairing",
    "pov": "point of view",
    "rip": "rest in peace",
    "smh": "shaking my head",
    "tb": "throwback",
    "tldr": "too long; didn’t read",
    "ty": "thank you",
    "tysm": "thank you so much",
    "v": "very",
    "w/": "with",
    "w/o": "without",
    "xD": "laughing / happy face",
    "yass": "yes",
    "zzz": "sleeping / bored"
}

In [27]:
def chat_conversion(text):
    new_text=[]
    for i in text.split():
        if i.lower() in chat_dict:
            new_text.append(chat_dict[i.lower()])
        else:
            new_text.append(i)
    return " ".join(new_text)
        

In [28]:
("fyi bh")

'fyi bh'

6.Spelling Correction


In [29]:
from textblob import TextBlob

In [30]:
incorrect_text="ceertain cinditions duriing several gennerations arre moodified in the saame maner."
textBlob=TextBlob(incorrect_text)
textBlob.correct().string

'certain conditions during several generations are modified in the same manner.'

7. Removing Stop words
like a,the,of,are,my etc

In [31]:
from nltk.corpus import stopwords

In [32]:
stopwords.words("english") # it can for any language .. like "spanish" etc

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [33]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words("english"):
            new_text.append('')
        else:
            new_text.append(word)
    x=new_text[:]
    new_text.clear()
    return " ".join(x)

In [34]:
words="i am fine" # where i and am are stopwords
remove_stopwords(words)

'  fine'

In [35]:
#df['review']=df['review'].apply(remove_stopwords)

In [36]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,i am a catholic taught in parochial elementary...,negative
49998,im going to have to disagree with the previous...,negative


****8.Removing the emojis or substitute it with corresponding meaningful sentense.

          i.Removing the emojis

In [37]:
import re

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"  # other symbols
                           u"\U000024C2-\U0001F251"  # enclosed characters
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# Example usage:
sample_text = "Hello 👋, this is a test 🚀🔥!"
clean_text = remove_emoji(sample_text)
print(clean_text)   # Output: Hello , this is a test !


Hello , this is a test !


In [38]:
df['review']=df['review'].apply(remove_emoji)

       ii.# replacing the emojis with sentense

In [39]:
import emoji
print(emoji.demojize("python is 🔥"))
print(emoji.demojize("it is so🥵"))
# using this function we can substitute emojis as sentense

python is :fire:
it is so:hot_face:


# 9.Tokenization


at the time of tokenization these 4 challenges occur.
Prefix: Character(s) at the beginning
Examples: $ ( " ¿

Suffix: Character(s) at the end
Examples: km ) , . ! ”

Infix: Character(s) in between
Examples: - -- / ...

Exception: Special-case rule to split a string into several tokens or prevent a token from being split when punctuation rules are applied
Examples: let’s U.S.

# 9.i.Using the split function
 example: i am going to varendra University. 
 (if we split it according to space it will tokenized. It is called word tokenization)

 Example:I am going to varendra University. I will be back as early as possible. 
(if we split it according to "." it will tokenized. It is called sentense tokenization.)
 

 

In [40]:
#word tokenization
sentense1="i am going to varendra University"
sentense1.split(" ")

['i', 'am', 'going', 'to', 'varendra', 'University']

In [41]:
#sentense tokenization
sent2="I am going to varendra University.I will be back as early as possible."
sent2.split(".")

['I am going to varendra University',
 'I will be back as early as possible',
 '']

# limitation of split function

In [42]:
sent2="i am going to varendra University!"
sent2.split(" ")
# where last word is university! but it is different (University).so right tokenization not happened. 

['i', 'am', 'going', 'to', 'varendra', 'University!']

In [43]:
sent3="where do you think i should go? I have 3 day holiday"
sent3.split(".")
#in this example we can see if we want to split with "."to acheive sentense tokenization it will wrong
#bacause here sentense should differentiate with respect to question(?) mark
# so we cant able to handle this using split  bacause we can split anything based on one thing at a time

['where do you think i should go? I have 3 day holiday']

# 9.ii Regular Expression

In [44]:
import re
sent3="i am going to University!"
tokens=re.findall("[\w']+",sent3)
tokens


['i', 'am', 'going', 'to', 'University']

In [45]:

text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry?Lorem Ipsum has been the industry's standard dummy text ever since the 1500s,when an unknown printer took a galley of type and scrambled it to make a type specimen book."
 
sentences=re.compile('[.!?]').split(text) 
sentences

['Lorem Ipsum is simply dummy text of the printing and typesetting industry',
 "Lorem Ipsum has been the industry's standard dummy text ever since the 1500s,when an unknown printer took a galley of type and scrambled it to make a type specimen book",
 '']

# 9.iii.NLTK

In [46]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [47]:
sent1="i am going to varendra University!"
word_tokenize(sent1)

['i', 'am', 'going', 'to', 'varendra', 'University', '!']

In [48]:
sent4="where do you think i should go? I have 3 day holiday"
sent_tokenize(sent4)

['where do you think i should go?', 'I have 3 day holiday']

In [49]:
sent5="I have a Ph.D in A.I"
sent6="we're here to help! mail us any@gmail.com"
sent7="A 5km ride cost $10.00"
word_tokenize(sent5)




['I', 'have', 'a', 'Ph.D', 'in', 'A.I']

In [50]:
word_tokenize(sent6)

['we', "'re", 'here', 'to', 'help', '!', 'mail', 'us', 'any', '@', 'gmail.com']

In [51]:
word_tokenize(sent7)

['A', '5km', 'ride', 'cost', '$', '10.00']

#  9.iv. Spacy
##  it  works most perfectly for complex data otherwise NlTK which perform well for less complex data

In [52]:
import spacy
nlp=spacy.load("en_core_web_sm")
#nlp = spacy.load("en_core_web_sm")       # load small English model
#nlp = spacy.load("en_core_web_md")       # medium model
#nlp = spacy.load("en_core_web_lg")       # large model
#nlp = spacy.load("/path/to/model")       # local path

In [53]:
# sentense need to documented like this
doc1=nlp(sent5)
doc2=nlp(sent6)
doc3=nlp(sent7)

for token in doc1:
    print(token)

I
have
a
Ph
.
D
in
A.I


In [54]:
for token in doc2:
    print(token)

we
're
here
to
help
!
mail
us
any@gmail.com


In [55]:
for token in doc3:
    print(token)

A
5
km
ride
cost
$
10.00


# 10.Stemming

Stemming(Inflection:inflection is the modification of a word to express different grammatical categories such as tense,case,voice,aspect,person,number,gender and mood. like walk can be modify as walking,walked,walks.. this is called inflection

Now stemming is the process to reduce inflection.. simply to find the root words. for above example we have to extract root word walk from walks,walking,walked etc..

this is used in IR System( information retrieval system).
 when we search fish it will also show some content about fishing and other fish related content.. if this not happed that indicates a bad system.

# using Nltk

In [56]:
from nltk.stem.porter import PorterStemmer

In [57]:
ps=PorterStemmer()
def stemming_word(text):
    return " ".join([ps.stem(word) for word in text.split()])
    

In [58]:
sample=" fish fishing "
stemming_word(sample)


'fish fish'

In [59]:
sample="walk walks walking walked"
stemming_word(sample)

'walk walk walk walk'

In [60]:
df['review']=df['review'].apply(stemming_word)

# 11.Lemmatization

In [61]:
import nltk
from nltk.stem import WordNetLemmatizer
Word_net_lemmatizer=WordNetLemmatizer()

sample="He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun"
punctuations="?:!.,;"
# first tokenized the sentense into word
tokenized_words=nltk.word_tokenize(sample)
for word in tokenized_words:
    if word in punctuations:
        tokenized_words.remove(word)

tokenized_words
print ("{0:20}{1:20}".format("Word","Lemma"))
for word in tokenized_words:
    print ("{0:20}{1:20}".format(word,Word_net_lemmatizer.lemmatize(word)))  
    
     
     


Word                Lemma               
He                  He                  
was                 wa                  
running             running             
and                 and                 
eating              eating              
at                  at                  
same                same                
time                time                
He                  He                  
has                 ha                  
bad                 bad                 
habit               habit               
of                  of                  
swimming            swimming            
after               after               
playing             playing             
long                long                
hours               hour                
in                  in                  
the                 the                 
Sun                 Sun                 


In [62]:
# if we want Lemma as verb we have to specify pos(parts of speech ) as 'v' then change will notice
import nltk
from nltk.stem import WordNetLemmatizer
Word_net_lemmatizer=WordNetLemmatizer()

sample="He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun"
punctuations="?:!.,;"
# first tokenized the sentense into word
tokenized_words=nltk.word_tokenize(sample)
for word in tokenized_words:
    if word in punctuations:
        tokenized_words.remove(word)

tokenized_words
print ("{0:20}{1:20}".format("Word","Lemma"))
for word in tokenized_words:
    print ("{0:20}{1:20}".format(word,Word_net_lemmatizer.lemmatize(word,pos='v')))  
    
     
     


Word                Lemma               
He                  He                  
was                 be                  
running             run                 
and                 and                 
eating              eat                 
at                  at                  
same                same                
time                time                
He                  He                  
has                 have                
bad                 bad                 
habit               habit               
of                  of                  
swimming            swim                
after               after               
playing             play                
long                long                
hours               hours               
in                  in                  
the                 the                 
Sun                 Sun                 
